## 第一步 归因分析

In [ ]:
import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy
import datetime
import math
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')
# 正常显示中文
plt.rcParams['font.sans-serif']=['SimHei']
# 正常显示符号
plt.rcParams['axes.unicode_minus']=False
sql_engine = sqlalchemy.create_engine(
#这里是公司内部的数据库无法共享
    ), poolclass=sqlalchemy.pool.NullPool

connection = sql_engine.connect()

In [ ]:
def build_financial_samples(years, connection):
    """
    构建跨年度估值-财务合并样本，包括同比增速。
    
    参数：
        years: 列表，如 [2015, 2016, ..., 2023]
        connection: 数据库连接对象（SQLAlchemy）
    
    返回：
        DataFrame，包含估值、财务数据与同比增速的合并样本
    """
    import pandas as pd

    samples = []

    for y in years:
        start = f"{y}-04-30"
        end = f"{y+1}-04-30"

        # 当前年度估值数据
        val = pd.read_sql(f'''
            SELECT TRADE_CODE, DT, PE_TTM_PCT_RANK, PE_TTM, PB_LF, MKT_CAP_ARD
            FROM stockmarketvaluation
            WHERE DT = (
                SELECT MIN(DT) FROM stockmarketvaluation
                WHERE DT >= "{start}"
            )
        ''', connection, parse_dates=['DT'])

        # 本年度财务数据
        now_start = f"{y-1}-04-30"
        now_end = f"{y}-04-30"
        fin = pd.read_sql(f'''
            SELECT TRADE_CODE, NP_BELONGTO_PARCOMSH, TOT_OPER_REV, DT AS YEAR_REPORT_DATE
            FROM financialinfo
            WHERE MONTH(DT) = 12
            AND DT BETWEEN "{now_start}" AND "{now_end}"
        ''', connection, parse_dates=['YEAR_REPORT_DATE'])

        # 上年度财务数据
        last_start = f"{y-2}-04-30"
        last_end = f"{y-1}-04-30"
        fin_last = pd.read_sql(f'''
            SELECT TRADE_CODE, NP_BELONGTO_PARCOMSH, TOT_OPER_REV, DT AS YEAR_REPORT_DATE
            FROM financialinfo
            WHERE MONTH(DT) = 12
            AND DT BETWEEN "{last_start}" AND "{last_end}"
        ''', connection, parse_dates=['YEAR_REPORT_DATE'])

        # 去重：保留每个代码最新年报
        fin = fin.sort_values(['TRADE_CODE', 'YEAR_REPORT_DATE'], ascending=[True, False]).drop_duplicates('TRADE_CODE')
        fin_last = fin_last.sort_values(['TRADE_CODE', 'YEAR_REPORT_DATE'], ascending=[True, False]).drop_duplicates('TRADE_CODE')

        # 合并计算同比
        fin_merged = pd.merge(fin, fin_last, on='TRADE_CODE', suffixes=('_this_year', '_last_year'))
        fin_merged['归母净利润_YOY'] = (
            (fin_merged['NP_BELONGTO_PARCOMSH_this_year'] - fin_merged['NP_BELONGTO_PARCOMSH_last_year']) /
            fin_merged['NP_BELONGTO_PARCOMSH_last_year'] * 100
        )
        fin_merged['营业总收入_YOY'] = (
            (fin_merged['TOT_OPER_REV_this_year'] - fin_merged['TOT_OPER_REV_last_year']) /
            fin_merged['TOT_OPER_REV_last_year'] * 100
        )

        # 合并估值与财务增速
        merged = pd.merge(val, fin_merged, on='TRADE_CODE', how='left')
        merged['sample_year'] = f"{y}-{y+1}"
        samples.append(merged)

    return pd.concat(samples, ignore_index=True)


In [ ]:
def filter_by_market_cap(samples, drop_percent=0.3):
    """
    根据市值过滤掉市值最小的一部分股票，并保留需要字段。

    参数：
        samples: 包含市值列的DataFrame
        drop_percent: 要剔除的底部比例（如0.3表示剔除市值最低的30%）

    返回：
        过滤后保留字段的DataFrame
    """
    sorted_samples = samples.sort_values(by='MKT_CAP_ARD', ascending=False)
    threshold = sorted_samples['MKT_CAP_ARD'].quantile(drop_percent)
    filtered = sorted_samples[sorted_samples['MKT_CAP_ARD'] > threshold]

    selected_columns = [
        'TRADE_CODE', 'DT', 'PE_TTM_PCT_RANK',
        '归母净利润_YOY', '营业总收入_YOY',
        'PE_TTM', 'PB_LF'
    ]
    return filtered[selected_columns]


In [ ]:
def calculate_3y_rolling_roe(years, connection):
    """
    计算给定年份序列的每个年份对应前3年ROE的均值与标准差。

    参数：
        years: 如 [2015, 2016, ..., 2023]
        connection: SQLAlchemy 数据库连接对象

    返回：
        DataFrame，包含TRADE_CODE、ROE_3Y_AVG、ROE_3Y_STD、year字段
    """
    import pandas as pd

    roe_by_year = []

    for y in years:
        prior_years = [y - 3, y - 2, y - 1]
        window_data = []

        for py in prior_years:
            query = f"""
            SELECT TRADE_CODE, DT, NP_BELONGTO_PARCOMSH, TOT_ASSETS, TOT_LIAB
            FROM financialinfo
            WHERE DAY(DT) = 31 AND MONTH(DT) = 12 AND YEAR(DT) = {py}
            """
            df = pd.read_sql(query, connection, parse_dates=['DT'])
            df = df.sort_values(['TRADE_CODE', 'DT'], ascending=[True, False])
            df = df.drop_duplicates(subset=['TRADE_CODE'])
            df = df.dropna(subset=['TOT_ASSETS', 'TOT_LIAB', 'NP_BELONGTO_PARCOMSH'])
            df = df[(df['TOT_ASSETS'] - df['TOT_LIAB']) != 0]
            df['ROE'] = df['NP_BELONGTO_PARCOMSH'] / (df['TOT_ASSETS'] - df['TOT_LIAB']) * 100
            df = df[(df['ROE'] >= -100) & (df['ROE'] <= 100)]
            df['year'] = py
            window_data.append(df[['TRADE_CODE', 'ROE', 'year']])

        merged = pd.concat(window_data)
        agg = merged.groupby('TRADE_CODE')['ROE'].agg(
            ROE_3Y_AVG='mean',
            ROE_3Y_STD='std'
        ).reset_index()
        agg['year'] = y
        roe_by_year.append(agg)

    return pd.concat(roe_by_year).sort_values(['TRADE_CODE', 'year']).reset_index(drop=True)


ROE计算

In [ ]:
def clean_roe_by_iqr(df, upper_clip=60.0):
    """
    使用 IQR 方法清洗 ROE_3Y_AVG 的异常值。

    参数：
        df: 包含 'ROE_3Y_AVG' 列的 DataFrame
        upper_clip: 设置最大上限值，防止极端值影响结果（如60）

    返回：
        cleaned_df: 清洗后的 DataFrame
        lower_bound, upper_bound: 异常值阈值区间
    """
    Q1 = df['ROE_3Y_AVG'].quantile(0.25)
    Q3 = df['ROE_3Y_AVG'].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = min(Q3 + 1.5 * IQR, upper_clip)

    print(f"设定的 ROE_3Y_AVG 阈值区间为：[{lower_bound:.2f}%, {upper_bound:.2f}%]")

    cleaned_df = df[
        (df['ROE_3Y_AVG'] >= lower_bound) &
        (df['ROE_3Y_AVG'] <= upper_bound)
    ].copy()

    print(f"清洗后剩余样本数：{len(cleaned_df)}，原始样本数：{len(df)}")

    return cleaned_df, lower_bound, upper_bound


In [ ]:
def compute_roe_stability_threshold(df, quantile=0.75):
    """
    计算 ROE_3Y_STD 的稳定性阈值（如上四分位）。

    参数：
        df: 包含 'ROE_3Y_STD' 列的 DataFrame
        quantile: 分位数阈值（默认 0.75）

    返回：
        阈值（float）
    """
    threshold = df['ROE_3Y_STD'].quantile(quantile)
    print(f"设定的 ROE 稳定性阈值：{threshold:.2f}")
    return threshold


In [ ]:
def merge_roe_to_samples(samples_df, roe_df, dropna=True, verbose=True):
    """
    将三年滚动 ROE 特征（均值 + 标准差）合并到样本数据中，并自动处理缺失值。

    参数：
        samples_df (pd.DataFrame): 股票估值/财务样本数据（含 TRADE_CODE、DT）
        roe_df (pd.DataFrame): 计算好的 ROE 指标（含 TRADE_CODE、year、ROE_3Y_AVG、ROE_3Y_STD）
        dropna (bool): 是否自动删除合并后的缺失行，默认 True
        verbose (bool): 是否打印缺失信息和处理结果

    返回：
        pd.DataFrame: 合并并清洗后的样本数据
    """
    samples_df = samples_df.copy()
    samples_df['year'] = samples_df['DT'].dt.year

    # 确保 TRADE_CODE 是字符串类型
    samples_df['TRADE_CODE'] = samples_df['TRADE_CODE'].astype(str)
    roe_df['TRADE_CODE'] = roe_df['TRADE_CODE'].astype(str)

    # 合并
    merged = pd.merge(
        samples_df,
        roe_df[['TRADE_CODE', 'year', 'ROE_3Y_AVG', 'ROE_3Y_STD']],
        on=['TRADE_CODE', 'year'],
        how='left'
    )

    if verbose:
        print("合并后缺失统计：")
        print(merged.isnull().sum())

    if dropna:
        merged = merged.dropna()
        if verbose:
            print(f"缺失值已删除，剩余样本数：{len(merged)}")

    return merged


高估值成长股和低估值价值股的分类


In [ ]:
def classify_and_label_stocks(merged_data, stability_threshold):
    """
    对股票进行分类标记：
    - 低估值 + 稳定ROE（label=0）
    - 高估值 + 高增长（label=1）
    - 其他股票（label=-1）

    参数：
        merged_data (pd.DataFrame): 合并后的估值与财务数据
        stability_threshold (float): ROE_3Y_STD 的稳定性阈值

    返回：
        merged_class (pd.DataFrame): 包含 TRADE_CODE, DT, 分类标签的 DataFrame
    """
    df = merged_data.copy()

    # 定义分类逻辑
    df['is_undervalued'] = (
        ((df['PE_TTM'] < 10) & (df['PE_TTM'] > 0)) | 
        ((df['PB_LF'] < 1) & (df['PE_TTM'] < 30))
    )
    df['is_stable_roe'] = df['ROE_3Y_STD'] < stability_threshold
    df['is_undervalued_and_stable_roe'] = df['is_undervalued'] & df['is_stable_roe']

    df['is_high_valued'] = df['PE_TTM_PCT_RANK'] > 0.7
    df['is_high_growth'] = (df['归母净利润_YOY'] > 20) & (df['营业总收入_YOY'] > 20)
    df['is_high_valued_growth_stock'] = df['is_high_valued'] & df['is_high_growth']

    # # 构造分类与标签列
    # selected_columns = [
    #     'TRADE_CODE', 'DT', 'is_undervalued_and_stable_roe',
    #     'is_high_valued_growth_stock'
    # ]
    merged_class = df.copy()

    # 生成 sample_year
    merged_class['sample_year'] = (
        merged_class['DT'].dt.year.astype(str) + '-' + (merged_class['DT'].dt.year + 1).astype(str)
    )

    # 标签标注
    merged_class['label'] = -1
    merged_class.loc[merged_class['is_high_valued_growth_stock'], 'label'] = 1
    merged_class.loc[merged_class['is_undervalued_and_stable_roe'], 'label'] = 0

    return merged_class


2021-04-30到2022-04-30区间的真正收盘价

In [ ]:
def calculate_relative_stock_returns(years, connection, index_code='881001.WI'):
    """
    计算每只股票每年相对于指数的超额收益率。

    参数：
        years (list[int]): 样本年份列表（如 [2015, ..., 2023] 表示区间 2015~2024）
        connection: 数据库连接对象
        index_code (str): 指数代码（默认中证全指 881001.WI）

    返回：
        all_marketinfo (DataFrame): 包含每年个股收益与相对收益
        all_indexprice (DataFrame): 每年指数收益
    """
    import pandas as pd

    marketinfo_samples = []
    indexprice_samples = []

    for y in years:
        start = f"{y}-04-30"
        end = f"{y+1}-04-30"

        # 个股市场数据
        query_marketinfo = f"""
        SELECT TRADE_CODE, DT, CLOSE, FACTOR
        FROM marketinfo
        WHERE DT IN (
            SELECT MIN(DT) FROM marketinfo WHERE DT >= '{start}'
            UNION
            SELECT MIN(DT) FROM marketinfo WHERE DT >= '{end}'
        )
        """
        marketinfo_data = pd.read_sql(query_marketinfo, connection, parse_dates=['DT'])
        marketinfo_data['TRUE_CLOSE'] = marketinfo_data['CLOSE'] * marketinfo_data['FACTOR']
        marketinfo_data['year'] = marketinfo_data['DT'].dt.year

        # 按年份拆分
        close_y = marketinfo_data[marketinfo_data['year'] == y].sort_values('DT').groupby('TRADE_CODE').last().reset_index()
        close_y1 = marketinfo_data[marketinfo_data['year'] == y + 1].sort_values('DT').groupby('TRADE_CODE').last().reset_index()
        close_y = close_y[['TRADE_CODE', 'TRUE_CLOSE']].rename(columns={'TRUE_CLOSE': 'CLOSE_Y'})
        close_y1 = close_y1[['TRADE_CODE', 'TRUE_CLOSE']].rename(columns={'TRUE_CLOSE': 'CLOSE_Y1'})

        stock_perf = pd.merge(close_y, close_y1, on='TRADE_CODE')
        stock_perf['STOCK_CHANGE'] = (stock_perf['CLOSE_Y1'] - stock_perf['CLOSE_Y']) / stock_perf['CLOSE_Y'] * 100

        # 指数数据
        query_index = f"""
        SELECT TRADE_CODE, DT, CLOSE, FACTOR
        FROM indexcloseprice
        WHERE TRADE_CODE = '{index_code}'
        AND DT IN (
            SELECT MIN(DT) FROM indexcloseprice WHERE TRADE_CODE = '{index_code}' AND DT >= '{start}'
            UNION
            SELECT MIN(DT) FROM indexcloseprice WHERE TRADE_CODE = '{index_code}' AND DT >= '{end}'
        )
        """
        index_data = pd.read_sql(query_index, connection, parse_dates=['DT'])
        index_data['TRUE_CLOSE'] = index_data['CLOSE'] * index_data['FACTOR']
        index_data['year'] = index_data['DT'].dt.year

        idx_y = index_data[index_data['year'] == y].sort_values('DT').groupby('TRADE_CODE').last().reset_index()
        idx_y1 = index_data[index_data['year'] == y + 1].sort_values('DT').groupby('TRADE_CODE').last().reset_index()
        idx_y = idx_y[['TRADE_CODE', 'TRUE_CLOSE']].rename(columns={'TRUE_CLOSE': 'CLOSE_Y'})
        idx_y1 = idx_y1[['TRADE_CODE', 'TRUE_CLOSE']].rename(columns={'TRUE_CLOSE': 'CLOSE_Y1'})

        idx_perf = pd.merge(idx_y, idx_y1, on='TRADE_CODE')
        idx_perf['STOCK_CHANGE'] = (idx_perf['CLOSE_Y1'] - idx_perf['CLOSE_Y']) / idx_perf['CLOSE_Y'] * 100

        # 计算超额收益
        index_return = idx_perf['STOCK_CHANGE'].values[0]
        stock_perf['RELATIVE_TO_MARKET'] = stock_perf['STOCK_CHANGE'] - index_return

        # 添加 sample_year 标记
        stock_perf['sample_year'] = f"{y}-{y+1}"
        idx_perf['sample_year'] = f"{y}-{y+1}"

        marketinfo_samples.append(stock_perf)
        indexprice_samples.append(idx_perf)

    all_marketinfo = pd.concat(marketinfo_samples).reset_index(drop=True)
    all_indexprice = pd.concat(indexprice_samples).reset_index(drop=True)

    return all_marketinfo, all_indexprice


In [ ]:
def merge_stock_return_with_labels(all_marketinfo, merged_class, dropna=True, verbose=True):
    """
    将收益率数据与分类标签合并，并构建最终监督学习数据集。

    参数：
        all_marketinfo (DataFrame): 包含每年股票收益率与相对收益数据
        merged_class (DataFrame): 包含每年分类标签（label、分类逻辑）
        dropna (bool): 是否删除缺失值（默认 True）
        verbose (bool): 是否打印缺失统计信息

    返回：
        merged_xy (DataFrame): 包含 TRADE_CODE、sample_year、收益、分类标签的数据集
    """
    merged = pd.merge(
        all_marketinfo,
        merged_class,
        on=['TRADE_CODE', 'sample_year'],
        how='inner'
    )

    merged_xy = merged[[
        'TRADE_CODE', 'sample_year', 'STOCK_CHANGE', 'RELATIVE_TO_MARKET',
        'is_undervalued_and_stable_roe', 'is_high_valued_growth_stock', 'label', '归母净利润_YOY', '营业总收入_YOY','ROE_3Y_AVG', 'ROE_3Y_STD'
    ]].copy()

    if verbose:
        print("合并后缺失值统计：")
        print(merged_xy.isnull().sum())

    if dropna:
        merged_xy = merged_xy.dropna()
        if verbose:
            print(f"缺失值已删除，剩余样本数：{len(merged_xy)}")

    return merged_xy


高估值成长股 和 低估值价值股 再进行涨跌幅的前五十和后五十的排序 总共四类 涨的高估值成长股50只，跌的高估值成长股50只，涨的低估值价值股50只，跌的低估值价值股50只

In [ ]:
def select_extreme_performance_stocks(merged_xy, top_n=50, verbose=False):
    """
    根据相对收益对高估值成长股和低估值价值股进行排序，
    提取涨幅前/后N只股票，并标记涨跌。

    参数：
        merged_xy (DataFrame): 包含分类标签与收益的样本数据
        top_n (int): 提取涨跌幅前/后N只股票
        verbose (bool): 是否打印结果

    返回：
        combined_stocks (DataFrame): 合并后的代表性样本，保留所有原始列并添加涨跌标记
    """
    # 高估值成长股
    high_growth = merged_xy[merged_xy['is_high_valued_growth_stock'] == True]
    high_growth = high_growth.dropna(subset=['RELATIVE_TO_MARKET'])
    high_growth_sorted = high_growth.sort_values(by='RELATIVE_TO_MARKET', ascending=False)
    top_high_growth = high_growth_sorted.head(top_n).copy()
    bottom_high_growth = high_growth_sorted.tail(top_n).copy()

    top_high_growth['PERFORMANCE_TAG'] = '涨幅最大_高估值成长股'
    bottom_high_growth['PERFORMANCE_TAG'] = '跌幅最大_高估值成长股'

    # 低估值价值股
    value_stocks = merged_xy[merged_xy['is_undervalued_and_stable_roe'] == True]
    value_stocks = value_stocks.dropna(subset=['RELATIVE_TO_MARKET'])
    value_sorted = value_stocks.sort_values(by='RELATIVE_TO_MARKET', ascending=True)
    bottom_value = value_sorted.head(top_n).copy()
    top_value = value_sorted.tail(top_n).copy()

    top_value['PERFORMANCE_TAG'] = '涨幅最大_低估值价值股'
    bottom_value['PERFORMANCE_TAG'] = '跌幅最大_低估值价值股'

    # 输出（可选）
    if verbose:
        print("涨幅最大的高估值成长股：")
        print(top_high_growth)
        print("跌幅最小的高估值成长股：")
        print(bottom_high_growth)
        print("涨幅最大的低估值价值股：")
        print(top_value)
        print("跌幅最大的低估值价值股：")
        print(bottom_value)

    # 合并保留所有原始列 + 涨跌标记
    combined_stocks = pd.concat([
        top_high_growth,
        bottom_high_growth,
        top_value,
        bottom_value
    ], ignore_index=True)

    return combined_stocks


In [ ]:
years = list(range(2015, 2024))
all_samples = build_financial_samples(years, connection)

# 已经有 all_samples（从前面的 build_financial_samples 得到）
all_samples_selected = filter_by_market_cap(all_samples)

# 计算 3 年滚动 ROE
final_roe = calculate_3y_rolling_roe(years, connection)

# 清洗 ROE 平均值
cleaned_roe, lower, upper = clean_roe_by_iqr(final_roe)

# 获取稳定性阈值
stability_threshold = compute_roe_stability_threshold(cleaned_roe)
merged_data = merge_roe_to_samples(all_samples_selected, cleaned_roe)
print(merged_data.head())
merged_class = classify_and_label_stocks(merged_data, stability_threshold)
merged_0 = merged_class[merged_class['label'] == 0]
merged_1 = merged_class[merged_class['label'] == 1]

all_marketinfo, all_indexprice = calculate_relative_stock_returns(years, connection)
print(all_marketinfo.head())
merged_xy = merge_stock_return_with_labels(all_marketinfo, merged_class)
print(merged_xy.head(20))



In [ ]:
# 高估值成长股
num_high_growth = int(
    merged_xy[merged_xy['is_high_valued_growth_stock'] == True].shape[0]
)

# 低估值价值股
num_value_stock = int(
    merged_xy[merged_xy['is_undervalued_and_stable_roe'] == True].shape[0]
)

# 取两者最小值（或按其中一个一致）
top_n = min(num_high_growth, num_value_stock)

print(f"高估值成长股: {num_high_growth}, 低估值价值股: {num_value_stock}")
print(f"最终取前后数量: {top_n}")


In [ ]:
combined_stocks = select_extreme_performance_stocks(merged_xy, top_n=50)


In [ ]:
combined_financial_stocks = select_extreme_performance_stocks(merged_xy, top_n=800)


财务表

In [ ]:
import pandas as pd
import numpy as np

# ======= 1️⃣ 原始字段 =======
SELECTED_COLS = [
    'TRADE_CODE', 'DT',
    'NET_PROFIT_IS', 'TOT_ASSETS', 'TOT_LIAB',
    'OPER_REV', 'FIX_ASSETS_TOT', 'OPER_COST',
    'SELLING_DIST_EXP', 'GERL_ADMIN_EXP', 'FIN_EXP_IS',
    'NET_CASH_FLOWS_OPER_ACT', 'NET_CASH_FLOWS_FNC_ACT',
    'NET_CASH_FLOWS_INV_ACT',
    'RESEARCHANDDEVELOPMENTEXPENSES',
    'CASH_PAY_ACQ_CONST_FIOLTA'
]

# ======= 2️⃣ 派生率类因子 =======
FEATURE_COLS = [
    'ROE', '固定资产周转率', '毛利率',
    '销售费用率', '管理费用率', '财务费用率',
    '销售净利率', '资产周转率', '财务杠杆率',
    '经营现金流量率', '筹资现金流量率', '投资现金流量率',
    '研发费用率',
    '资本开支率',          # CapEx / 营业收入
    '资本开支/固定资产'   # CapEx / 固定资产
]

# ======= 3️⃣ 需要算增速的原值因子 =======
GROWTH_COLS = [
    'CASH_PAY_ACQ_CONST_FIOLTA',      # 资本开支原值
    'RESEARCHANDDEVELOPMENTEXPENSES'  # 研发费用原值
]

# ======= 4️⃣ 计算占比率 =======
def compute_financial_features(df):
    def safe_div(n, d):
        return n / np.where(d == 0, np.nan, d)

    df['ROE'] = safe_div(df['NET_PROFIT_IS'], df['TOT_ASSETS'] - df['TOT_LIAB'])
    df['固定资产周转率'] = safe_div(df['OPER_REV'], df['FIX_ASSETS_TOT'])
    df['毛利率'] = 1 - safe_div(df['OPER_COST'], df['OPER_REV'])
    df['销售费用率'] = safe_div(df['SELLING_DIST_EXP'], df['OPER_REV'])
    df['管理费用率'] = safe_div(df['GERL_ADMIN_EXP'], df['OPER_REV'])
    df['财务费用率'] = safe_div(df['FIN_EXP_IS'], df['OPER_REV'])
    df['销售净利率'] = safe_div(df['NET_PROFIT_IS'], df['OPER_REV'])
    df['资产周转率'] = safe_div(df['OPER_REV'], df['TOT_ASSETS'])
    df['财务杠杆率'] = safe_div(df['TOT_ASSETS'], df['TOT_ASSETS'] - df['TOT_LIAB'])
    df['经营现金流量率'] = safe_div(df['NET_CASH_FLOWS_OPER_ACT'], df['OPER_REV'])
    df['筹资现金流量率'] = safe_div(df['NET_CASH_FLOWS_FNC_ACT'], df['OPER_REV'])
    df['投资现金流量率'] = safe_div(df['NET_CASH_FLOWS_INV_ACT'], df['OPER_REV'])
    df['研发费用率'] = safe_div(df['RESEARCHANDDEVELOPMENTEXPENSES'], df['OPER_REV'])
    df['资本开支率'] = safe_div(df['CASH_PAY_ACQ_CONST_FIOLTA'], df['OPER_REV'])
    df['资本开支/固定资产'] = safe_div(df['CASH_PAY_ACQ_CONST_FIOLTA'], df['FIX_ASSETS_TOT'])

    return df

# ======= 5️⃣ 主函数 =======
def build_detailed_financial_samples(years, connection):
    # === 年份范围：含 y-1 做基期 ===
    years_needed = list(set(years + [y-1 for y in years]))
    report_dates = [f"{y}-12-31" for y in years_needed]

    # === 拉数据 ===
    cols_str = ', '.join(SELECTED_COLS)
    dates_str = ', '.join([f"'{d}'" for d in report_dates])
    query = f"""
    SELECT {cols_str}
    FROM financialinfo
    WHERE DT IN ({dates_str})
    """
    df = pd.read_sql(query, connection, parse_dates=['DT'])

    # === 计算占比率 ===
    df = compute_financial_features(df)

    # === 标记样本年度，只标今年 ===
    df['sample_year'] = None
    for y in years:
        df.loc[df['DT'] == f"{y}-12-31", 'sample_year'] = f"{y}-{y+1}"

    # === 拆分今年 vs 去年，拼宽表 ===
    df_this = df[df['sample_year'].notna()].copy()
    df_prev = df[['TRADE_CODE', 'DT'] + FEATURE_COLS + GROWTH_COLS].copy()
    df_prev['DT'] = df_prev['DT'] + pd.DateOffset(years=1)  # 对齐
    df_prev = df_prev.rename(columns={col: f"{col}_PREV" for col in FEATURE_COLS + GROWTH_COLS})

    df_merged = df_this.merge(df_prev, on=['TRADE_CODE', 'DT'], how='left')

    # === 占比率 ===
    for col in FEATURE_COLS:
        prev, diff = f"{col}_PREV", f"{col}_DIFF"
        df_merged[diff] = np.where(df_merged[prev] == 0, np.nan, df_merged[col] - df_merged[prev])######

    # === 原值增速 ===
    for col in GROWTH_COLS:
        prev, yoy = f"{col}_PREV", f"{col}_YOY"
        df_merged[yoy] = np.where(df_merged[prev] == 0, np.nan, (df_merged[col] - df_merged[prev]) / df_merged[prev].abs())##yoy abs

    # === 最终要哪些列 ===
    keep_cols = (
        ['TRADE_CODE', 'DT', 'sample_year'] +
        FEATURE_COLS +
        [f"{col}_DIFF" for col in FEATURE_COLS] +
        GROWTH_COLS +
        [f"{col}_YOY" for col in GROWTH_COLS]
    )
    # === 对最终输出列统一改中文命名 ===
    rename_final_cols = {
        'CASH_PAY_ACQ_CONST_FIOLTA': '资本开支',
        'RESEARCHANDDEVELOPMENTEXPENSES': '研发费用',
        'CASH_PAY_ACQ_CONST_FIOLTA_YOY': '资本开支_GROWTH',
        'RESEARCHANDDEVELOPMENTEXPENSES_YOY': '研发费用_GROWTH'
    }

    # 返回时只改展示的列名，不动 df_merged
    return df_merged[keep_cols].rename(columns=rename_final_cols)


In [ ]:
# 用法示例
years = list(range(2015, 2024))
df_samples = build_detailed_financial_samples(years, connection)

print(df_samples.head())


In [ ]:
import pandas as pd

# # 先选出需要用到的列（去重防止重复）
# combined_cols = ['TRADE_CODE', 'sample_year',
#                  'RELATIVE_TO_MARKET',
#                  'PERFORMANCE_TAG',
#                  'label']

# combined_subset = combined_financial_stocks[combined_cols].drop_duplicates()

# 用 merge 保留完全匹配 + 附加需要的列
filtered_df = df_samples.merge(
    combined_financial_stocks,
    on=['TRADE_CODE', 'sample_year'],
    how='inner',
)
# 对带 `_y` 后缀的列进行重命名，去掉后缀
filtered_df = filtered_df.rename(columns=lambda col: col.rstrip('_y') if col.endswith('_y') else col)

# 检查结果
print(filtered_df.head())


In [ ]:
filtered_df.columns

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

def compute_feature_importance_by_style(
    df,
    style_tag,          # '高估值成长股' 或 '低估值价值股'
    drop_cols=None,     # 额外要丢的列
    random_state=42,
    n_estimators=100,
    plot_top_n=10
):
    """
    对指定风格做二分类特征重要性分析（涨跌标签来自 RELATIVE_TO_MARKET）
    
    参数:
        df : DataFrame, 必须含有 PERFORMANCE_TAG 和 RELATIVE_TO_MARKET
        style_tag : str, 要匹配的风格关键词
        drop_cols : list, 要丢弃的列（比如 ID、时间）
        random_state : int
        n_estimators : int, 随机森林棵树数
        plot_top_n : int, 是否绘制前N特征的重要性柱状图
    返回:
        feature_importance_df : DataFrame
    """
    # ========== 1. 创建涨跌标签 ==========
    df = df.copy()
    df['涨跌_label'] = df['RELATIVE_TO_MARKET'].apply(lambda x: 1 if x > 0 else 0)
    
    # ========== 2. 按风格分组 ==========
    subset_df = df[df['PERFORMANCE_TAG'].str.contains(style_tag)].copy()
    
    print(f"\n[{style_tag}] 样本数: {subset_df.shape[0]}")
    print(subset_df['涨跌_label'].value_counts())
    
    # ========== 3. 准备特征和标签 ==========
    drop_cols = drop_cols or []
    drop_cols_final = drop_cols + [
        'RELATIVE_TO_MARKET',
        'label',  # 风格标签
        '涨跌_label',
        'PERFORMANCE_TAG',
        'STOCK_CHANGE'
    ]
    
    X = subset_df.drop(columns=[col for col in drop_cols_final if col in subset_df.columns])
    y = subset_df['涨跌_label']
    
    # 填补缺失值
    X = X.fillna(X.mean())
    
    # ========== 4. 训练模型 ==========
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        random_state=random_state,
        class_weight='balanced'
    )
    clf.fit(X, y)
    
    # ========== 5. 特征重要性 ==========
    importances = clf.feature_importances_
    feature_importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': importances
    }).sort_values(by='importance', ascending=False).reset_index(drop=True)
    
    print(feature_importance_df.head(plot_top_n))
    
    # ========== 6. 可视化 ==========
    if plot_top_n:
        plt.figure(figsize=(8, 6))
        plt.barh(
            feature_importance_df['feature'].head(plot_top_n)[::-1],
            feature_importance_df['importance'].head(plot_top_n)[::-1]
        )
        plt.xlabel('Importance')
        plt.title(f'{style_tag} - Top {plot_top_n} Features')
        plt.show()
    
    return feature_importance_df


In [ ]:
filtered_df.columns


In [ ]:
import pandas as pd

# 定义要删除的静态列
static_columns = [
    'ROE', '固定资产周转率', '毛利率', '销售费用率', '管理费用率', '财务费用率', 
    '销售净利率', '资产周转率', '财务杠杆率', '经营现金流量率', '筹资现金流量率', 
    '投资现金流量率', '研发费用率', '资本开支率', '资本开支/固定资产',  '资本开支', '研发费用'
]

# 定义要保留的动态列和其他重要列
dynamic_columns = ['ROE_DIFF', '固定资产周转率_DIFF',
       '毛利率_DIFF', '销售费用率_DIFF', '管理费用率_DIFF', '财务费用率_DIFF', '销售净利率_DIFF',
       '资产周转率_DIFF', '财务杠杆率_DIFF', '经营现金流量率_DIFF', '筹资现金流量率_DIFF',
       '投资现金流量率_DIFF', '研发费用率_DIFF', '资本开支率_DIFF', '资本开支/固定资产_DIFF', '资本开支_GROWTH', '研发费用_GROWTH', '归母净利润_YOY', '营业总收入_YOY'
]

# 选择保留的其他列
important_columns = [
    'TRADE_CODE', 'DT', 'sample_year',
    'STOCK_CHANGE', 'RELATIVE_TO_MARKET', 'is_undervalued_and_stable_roe', 'is_high_valued_growth_stock',
    'label', 'ROE_3Y_AVG', 'ROE_3Y_STD', 'PERFORMANCE_TAG'
]

# 合并保留的列
columns_to_keep = dynamic_columns + important_columns

# 删除静态列，保留所需的列
filtered_df = filtered_df[columns_to_keep]

# 检查结果
print(filtered_df.head())


In [ ]:

drop_cols = [
    'TRADE_CODE', 'DT', 'sample_year',
    'YEAR_REPORT_DATE_this_year', 'YEAR_REPORT_DATE_last_year'  # 添加日期时间列
]

# 高估值成长股
growth_importance = compute_feature_importance_by_style(
    filtered_df,
    style_tag='高估值成长股',
    drop_cols=drop_cols,
    n_estimators=100,
    plot_top_n=20
)

# 低估值价值股
value_importance = compute_feature_importance_by_style(
    filtered_df,
    style_tag='低估值价值股',
    drop_cols=drop_cols,
    n_estimators=100,
    plot_top_n=20
)

In [ ]:
import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)

In [ ]:
# 1️⃣ 先把特征重要性 DataFrame 取出来
# 比如你有 growth_importance 和 value_importance

# 设置阈值
THRESHOLD = 0.05

# 高估值成长股
growth_top_features = growth_importance.loc[
    growth_importance['importance'] > THRESHOLD, 'feature'
].tolist()

print("高估值成长股核心特征:", growth_top_features)

# 低估值价值股
value_top_features = value_importance.loc[
    value_importance['importance'] > THRESHOLD, 'feature'
].tolist()

print("低估值价值股核心特征:", value_top_features)


合成管理层text

In [ ]:
def merge_meeting_content_exactly(filtered_df, connection):
    """
    从两个来源：
      - report_extract_text (extract_text)
      - cninfo (meeting_content)
    精确按 TRADE_CODE + 年份合并，生成 combined_content.

    参数：
        filtered_df: 需要包含 TRADE_CODE 和 sample_year 的 DataFrame
        connection: 数据库连接

    返回：
        合并后的 DataFrame，带 combined_content 列
    """
    import pandas as pd
    import re

    df = filtered_df.copy()

    # ========== 1️⃣ 从 report_extract_text 读取报告要点 ==========
    report_text = pd.read_sql("""
        SELECT stock_code, extract_text, report_title, publish_time
        FROM report_extract_text
    """, connection)

    # 清理空值
    report_text = report_text.dropna(subset=['stock_code', 'report_title'])

    # 提取报告年份
    report_text['year'] = report_text['report_title'].str.extract(r'(\d{4})')
    report_text = report_text.dropna(subset=['year'])
    report_text['year'] = report_text['year'].astype(int)

    # ========== 2️⃣ 从 cninfo 读取会议纪要 ==========
    meeting = pd.read_sql("""
        SELECT stock_code, article_title, meeting_content
        FROM cninfo
    """, connection)

    meeting = meeting.dropna(subset=['stock_code', 'article_title', 'meeting_content'])
    meeting['year'] = meeting['article_title'].str.extract(r'(\d{4})')
    meeting = meeting.dropna(subset=['year'])
    meeting['year'] = meeting['year'].astype(int)

    # 按股票 + 年聚合多条会议纪要
    meeting_grouped = meeting.groupby(['stock_code', 'year']).agg(
        all_meeting_content=('meeting_content', ' | '.join)
    ).reset_index()

    # ========== 3️⃣ 样本期准备 ==========
    df['start_year'] = df['sample_year'].str.split('-').str[0].astype(int)
    df['stock_code'] = df['TRADE_CODE'].apply(lambda x: re.sub(r'\.(SH|SZ|BJ|HK)$', '', x.strip()))

    # ========== 4️⃣ 合并报告要点 ==========
    df = df.merge(
        report_text[['stock_code', 'year', 'extract_text']],
        left_on=['stock_code', 'start_year'],
        right_on=['stock_code', 'year'],
        how='left'
    )

    # ========== 5️⃣ 合并会议纪要 ==========
    df = df.merge(
        meeting_grouped[['stock_code', 'year', 'all_meeting_content']],
        left_on=['stock_code', 'start_year'],
        right_on=['stock_code', 'year'],
        how='left',
        suffixes=('', '_meeting')  # 避免列名冲突
    )

    # ========== 6️⃣ 拼接成 combined_content ==========
    df['extract_text'] = df['extract_text'].fillna(' ')
    df['all_meeting_content'] = df['all_meeting_content'].fillna(' ')
    df['combined_content'] = df['extract_text'] + ' ' + df['all_meeting_content']

    # 可选：去除多余列
    df.drop(columns=['year', 'year_meeting'], errors='ignore', inplace=True)

    # 检查输出
    print(df[['TRADE_CODE', 'sample_year', 'combined_content']].head())

    return df


In [ ]:
print(filtered_df.columns.tolist())


In [ ]:
combined_financial_stocks = merge_meeting_content_exactly(filtered_df, connection)


In [ ]:
combined_financial_stocks

In [ ]:
combined_stocks

In [ ]:
# 先准备 key
key_cols = ['TRADE_CODE', 'sample_year']

# 保留左表：只留 key 和需要的列
cols_to_keep = ['TRADE_CODE', 'sample_year'] + [
    col for col in combined_stocks.columns if col not in combined_financial_stocks.columns
]
merged = combined_stocks[cols_to_keep].merge(
    combined_financial_stocks,
    on=key_cols,
    how='inner'
)


In [ ]:
merged = combined_financial_stocks

In [ ]:
# # 保存路径 & 文件名
# output_file = 'merged_financial_samples_1.xlsx'

# # 保存
# merged.to_excel(output_file, index=False)

# print(f"保存成功！文件已生成: {output_file}")


In [ ]:
#  label=1 是高估值成长股，label=0 是低估值价值股
growth_up   = merged[(merged['label'] == 1) & (merged['RELATIVE_TO_MARKET'] > 0)]
growth_down = merged[(merged['label'] == 1) & (merged['RELATIVE_TO_MARKET'] < 0)]
value_up    = merged[(merged['label'] == 0) & (merged['RELATIVE_TO_MARKET'] > 0)]
value_down  = merged[(merged['label'] == 0) & (merged['RELATIVE_TO_MARKET'] < 0)]


In [ ]:
def analyze_group(df_subset, features, name):
    desc = df_subset[features].describe().T[['mean', 'std']]
    print(f"\n{name} 样本数: {df_subset.shape[0]}")
    print(desc)
    return desc


In [ ]:

# 高估值成长股 上涨
growth_up_desc = analyze_group(growth_up, growth_top_features, "高估值成长股-上涨")

# 高估值成长股 下跌
growth_down_desc = analyze_group(growth_down, growth_top_features, "高估值成长股-下跌")

# 低估值价值股 上涨
value_up_desc = analyze_group(value_up, value_top_features, "低估值价值股-上涨")

# 低估值价值股 下跌
value_down_desc = analyze_group(value_down, value_top_features, "低估值价值股-下跌")


In [ ]:
def get_basicinfo_all():
    # 提取所有股票的最新行业分类
    basicinfo = pd.read_sql("""
        SELECT TRADE_CODE, INDUSTRY_SW_I
        FROM basicinfo
        WHERE DT = (SELECT MAX(DT) FROM basicinfo)
    """, connection)
    return basicinfo


In [ ]:
# 获取行业分类数据
industry_info = get_basicinfo_all()


In [ ]:

combined_financial_stocks = combined_financial_stocks.merge(
    industry_info, 
    on='TRADE_CODE', 
    how='left'
)


In [ ]:
def analyze_by_group_and_industry(df, label, direction, features, group_name, sort_by='行业'):
    """
    df: DataFrame 包含财务数据
    label: 0（低估值价值股）或 1（高估值成长股）
    direction: +1（上涨）或 -1（下跌）
    features: 要分析的财务指标列表
    group_name: 当前分类名称（如 “高估值成长股-上涨”）
    sort_by: 排序字段，默认按行业名
    """
    subset = df[(df['label'] == label) & 
                ((df['RELATIVE_TO_MARKET'] > 0) if direction > 0 else (df['RELATIVE_TO_MARKET'] < 0))]

    print(f"\n▶ {group_name} 总样本数：{subset.shape[0]}")
    
    result = []
    grouped = subset.groupby('INDUSTRY_SW_I')

    for industry, group_df in grouped:
        if group_df.empty:
            continue
        desc = group_df[features].describe().T[['mean', 'std']].copy()
        desc['行业'] = industry
        desc['样本数'] = group_df.shape[0]
        desc['分类'] = group_name
        desc['指标'] = desc.index
        result.append(desc[['行业', '分类', '指标', '样本数', 'mean', 'std']])  # 控制列顺序

    if result:
        result_df = pd.concat(result, ignore_index=True)
        if sort_by:
            result_df.sort_values(by=sort_by, inplace=True)
        return result_df
    else:
        print(f"⚠️ {group_name} 无有效行业数据")
        return pd.DataFrame(columns=['行业', '分类', '指标', '样本数', 'mean', 'std'])


In [ ]:


g_up    = analyze_by_group_and_industry(combined_financial_stocks, label=1, direction=+1, features=growth_top_features, group_name='高估值成长股-上涨')
g_down  = analyze_by_group_and_industry(combined_financial_stocks, label=1, direction=-1, features=growth_top_features, group_name='高估值成长股-下跌')
v_up    = analyze_by_group_and_industry(combined_financial_stocks, label=0, direction=+1, features=value_top_features, group_name='低估值价值股-上涨')
v_down  = analyze_by_group_and_industry(combined_financial_stocks, label=0, direction=-1, features=value_top_features, group_name='低估值价值股-下跌')

# 合并
final_result = pd.concat([g_up, g_down, v_up, v_down], ignore_index=True)

# # 展示结果
# import ace_tools as tools; tools.display_dataframe_to_user(name="行业-分类分组指标统计", dataframe=final_result)


In [ ]:
final_result

In [ ]:
def analyze_distinguishing_indicators(final_result):
    """
    根据三种标准筛选能够明显区分涨跌的指标：
    1. 增长率指标（_GROWTH）：上涨 vs 下跌方向相反（正负）
    2. 差值指标（_DIFF）：绝对值或相对差异大
    3. 比率指标（如 ROE_3Y_AVG）：绝对值或相对差异大
    """

    # 分类指标（来自你提供的核心特征）
    ratio_indicators = ['ROE_3Y_AVG']
    growth_indicators = ['资本开支_GROWTH']
    diff_indicators = [
        '销售净利率_DIFF', 'ROE_DIFF', '毛利率_DIFF',
        '财务费用率_DIFF', '经营现金流量率_DIFF',
        '财务杠杆率_DIFF', '筹资现金流量率_DIFF',
        '资产周转率_DIFF', '固定资产周转率_DIFF'
    ]
    scale_indicators = []  # 可按需添加，如总资产等

    results = []

    # 获取所有行业和指标组合
    industries = final_result['行业'].unique()
    indicators = final_result['指标'].unique()

    for industry in industries:
        for indicator in indicators:
            industry_data = final_result[
                (final_result['行业'] == industry) & 
                (final_result['指标'] == indicator)
            ]

            if len(industry_data) < 4:
                continue

            try:
                g_up_mean = industry_data[industry_data['分类'] == '高估值成长股-上涨']['mean'].iloc[0]
                g_down_mean = industry_data[industry_data['分类'] == '高估值成长股-下跌']['mean'].iloc[0]
                v_up_mean = industry_data[industry_data['分类'] == '低估值价值股-上涨']['mean'].iloc[0]
                v_down_mean = industry_data[industry_data['分类'] == '低估值价值股-下跌']['mean'].iloc[0]
            except IndexError:
                continue

            up_mean = (g_up_mean + v_up_mean) / 2
            down_mean = (g_down_mean + v_down_mean) / 2

            indicator_type = ""
            is_distinguishing = False
            reason = ""

            if indicator in growth_indicators:
                indicator_type = "增长率指标"
                if (up_mean > 0 and down_mean < 0) or (up_mean < 0 and down_mean > 0):
                    is_distinguishing = True
                    reason = f"方向相反：上涨{up_mean:.3f} vs 下跌{down_mean:.3f}"
                else:
                    reason = f"方向相同：上涨{up_mean:.3f} vs 下跌{down_mean:.3f}"

            elif indicator in ratio_indicators:
                indicator_type = "比率指标"
                abs_diff = abs(up_mean - down_mean)
                relative_diff = abs_diff / (abs(up_mean) + abs(down_mean) + 1e-8) * 2
                if abs_diff > 0.05 or relative_diff > 0.3:
                    is_distinguishing = True
                    reason = f"差异显著：上涨{up_mean:.3f} vs 下跌{down_mean:.3f} (相对差异{relative_diff:.1%})"
                else:
                    reason = f"差异较小：上涨{up_mean:.3f} vs 下跌{down_mean:.3f} (相对差异{relative_diff:.1%})"

            elif indicator in diff_indicators:
                indicator_type = "差值指标"
                abs_diff = abs(up_mean - down_mean)
                relative_diff = abs_diff / (abs(up_mean) + abs(down_mean) + 1e-8) * 2
                if abs_diff > 0.05 or relative_diff > 0.3:
                    is_distinguishing = True
                    reason = f"差异显著：上涨{up_mean:.3f} vs 下跌{down_mean:.3f} (相对差异{relative_diff:.1%})"
                else:
                    reason = f"差异较小：上涨{up_mean:.3f} vs 下跌{down_mean:.3f} (相对差异{relative_diff:.1%})"

            elif indicator in scale_indicators:
                indicator_type = "规模相关指标"
                reason = "依赖公司规模，意义有限"

            else:
                indicator_type = "其他指标"
                abs_diff = abs(up_mean - down_mean)
                relative_diff = abs_diff / (abs(up_mean) + abs(down_mean) + 1e-8) * 2
                if abs_diff > 0.05 or relative_diff > 0.3:
                    is_distinguishing = True
                    reason = f"差异显著：上涨{up_mean:.3f} vs 下跌{down_mean:.3f} (相对差异{relative_diff:.1%})"
                else:
                    reason = f"差异较小：上涨{up_mean:.3f} vs 下跌{down_mean:.3f} (相对差异{relative_diff:.1%})"

            results.append({
                '行业': industry,
                '指标': indicator,
                '指标类型': indicator_type,
                '上涨均值': up_mean,
                '下跌均值': down_mean,
                '是否明显区分': is_distinguishing,
                '区分原因': reason,
                '高估值成长股_上涨': g_up_mean,
                '高估值成长股_下跌': g_down_mean,
                '低估值价值股_上涨': v_up_mean,
                '低估值价值股_下跌': v_down_mean
            })

    return pd.DataFrame(results)


In [ ]:
distinguishing_analysis = analyze_distinguishing_indicators(final_result)
output = distinguishing_analysis


调用AI

In [ ]:
import json
from openai import OpenAI

MODEL = "anthropic/claude-sonnet-4"
# 创建OpenAI客户端 使用自己的api
client = OpenAI(
    api_key='', 
    base_url=''
)

# 构建系统提示
system_prompt = """你是一个金融分析师"""


def ask_ai(question):
    # 发送请求
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    )
    if response is not None:
        res = response.choices[0].message.content
    else:
        print("Error: API response is None")
        res = "Error: No response from API"

    return res

In [ ]:

def _analyze_subset(data, label_desc, direction, ask_ai):
    print(data)

    results = []
    updown = "上涨" if direction == "top" else "下跌"

    # 🔑 选对用哪个特征列表
    if "高估值成长股" in label_desc:
        core_features = growth_top_features
    else:
        core_features = value_top_features

    for _, row in data.iterrows():
        content = row["combined_content"]

        # ✅ 财务数据摘要
        summary_parts = []
        for col in core_features:
            val = row.get(col, None)
            if pd.notnull(val):
                summary_parts.append(f"{col}={val:.2%}")  # 百分号格式化
        financial_summary = "，".join(summary_parts)

        q = (
            f"请深入分析以下这家{label_desc}公司在业务与财务层面的背景，重点探究其股价{updown}的深层次原因，"
            f"**不是停留在表面的财务变化（如某项指标上升），而是要结合纪要/财报识别公司做出的决策、行为、策略调整等具体事件**。\n\n"
            f"【公司财务简况】：{financial_summary}\n\n"
            f"【公司纪要及财报文字】:\n{content}\n\n"
            f"分析应包括以下三部分：\n"
            f"1）判断该公司所属行业，并结合该行业在{output}中的整体财务特征以及数据变化；\n"
            f"2）明确判断该公司是否符合行业内{updown}企业的典型财务特征（请一句话总结结论）；\n"
            f"3）从公司纪要、财报和核心财务特征中，**追溯三条具体的公司行为或事件（如某项投资、产品调整、战略收缩、管理动作等）**，"
            f"这些事件是导致财务变化进而推动股价{updown}的根本原因。\n\n"
            f"请以清晰、简练、结构化语言作答。"
        )

        ans = ask_ai(q)

        results.append({
            "stock_code": row["stock_code"],
            "sample_year": row["sample_year"],
            "three_points_reasons": ans
        })

    for r in results:
        print(f"分析结果 for stock {r['stock_code']} {r['sample_year']} (三点原因):")
        print(r["three_points_reasons"], "\n", "=" * 50, "\n", sep="")

    follow_up = f"请根据前面提到的{label_desc}的每只股{updown}的原因，分析它们之间是否存在共性？有的话请严谨地给出？\n\n{results}"
    print(ask_ai(follow_up), "\n", "=" * 50, "\n", sep="")



def analyze_stocks(combined_stocks, ask_ai):
    # 高估值成长股
    label_1 = combined_stocks[combined_stocks["label"] == 1]
    label_1_sorted = label_1.sort_values(by="RELATIVE_TO_MARKET", ascending=False)
    top_50_label_1 = label_1_sorted.head(50)
    bottom_50_label_1 = label_1_sorted.tail(50)

    # 低估值价值股
    label_0 = combined_stocks[combined_stocks["label"] == 0]
    label_0_sorted = label_0.sort_values(by="RELATIVE_TO_MARKET", ascending=False)
    top_50_label_0 = label_0_sorted.head(50)
    bottom_50_label_0 = label_0_sorted.tail(50)

    # 分析四类股票
    _analyze_subset(top_50_label_1, "高估值成长股", "top", ask_ai)
    _analyze_subset(bottom_50_label_1, "高估值成长股", "bottom", ask_ai)
    _analyze_subset(top_50_label_0, "低估值价值股", "top", ask_ai)
    _analyze_subset(bottom_50_label_0, "低估值价值股", "bottom", ask_ai)


In [ ]:
analyze_stocks(merged, ask_ai)

## 第二步 选股验证

In [ ]:
def build_financial_samples_q(years, connection):
    """
    构建以每年3月31日为基准点的估值+财务数据，包含同比增速。
    
    参数：
        years: 列表，如 [2025]
        connection: SQLAlchemy连接
    返回：
        DataFrame：估值 + 财务数据 + YoY 增长率
    """
    import pandas as pd

    samples = []

    for y in years:
        # ===== 估值数据（取 3月31日最近日） =====
        val = pd.read_sql(f'''
            SELECT TRADE_CODE, DT, PE_TTM_PCT_RANK, PE_TTM, PB_LF, MKT_CAP_ARD
            FROM stockmarketvaluation
            WHERE DT = (
                SELECT MIN(DT) FROM stockmarketvaluation
                WHERE DT >= "{y}-03-31"
            )
        ''', connection, parse_dates=['DT'])

        # ===== 财务数据：今年（y），上一年（y-1），按季度取值 =====
        fin_this = pd.read_sql(f'''
            SELECT TRADE_CODE, NP_BELONGTO_PARCOMSH, TOT_OPER_REV, DT AS YEAR_REPORT_DATE
            FROM financialinfo
            WHERE DT = "{y}-03-31"
        ''', connection, parse_dates=['YEAR_REPORT_DATE'])

        fin_last = pd.read_sql(f'''
            SELECT TRADE_CODE, NP_BELONGTO_PARCOMSH, TOT_OPER_REV, DT AS YEAR_REPORT_DATE
            FROM financialinfo
            WHERE DT = "{y-1}-03-31"
        ''', connection, parse_dates=['YEAR_REPORT_DATE'])

        # ===== 合并计算 YoY =====
        fin_merged = pd.merge(fin_this, fin_last, on='TRADE_CODE', suffixes=('_this_year', '_last_year'))

        fin_merged['归母净利润_YOY'] = (
            (fin_merged['NP_BELONGTO_PARCOMSH_this_year'] - fin_merged['NP_BELONGTO_PARCOMSH_last_year']) /
            fin_merged['NP_BELONGTO_PARCOMSH_last_year'] * 100
        )
        fin_merged['营业总收入_YOY'] = (
            (fin_merged['TOT_OPER_REV_this_year'] - fin_merged['TOT_OPER_REV_last_year']) /
            fin_merged['TOT_OPER_REV_last_year'] * 100
        )

        # ===== 合并估值与财务增长 =====
        merged = pd.merge(val, fin_merged, on='TRADE_CODE', how='left')
        merged['sample_year'] = f"{y}-{y+1}"

        samples.append(merged)

    return pd.concat(samples, ignore_index=True)


In [ ]:
def calculate_3y_rolling_roe_q(years, connection):
    """
    计算给定年份序列的每个年份对应前3年ROE的均值与标准差。

    参数：
        years: 如 [2015, 2016, ..., 2023]
        connection: SQLAlchemy 数据库连接对象

    返回：
        DataFrame，包含TRADE_CODE、ROE_3Y_AVG、ROE_3Y_STD、year字段
    """
    import pandas as pd

    roe_by_year = []

    for y in years:
        prior_years = [y - 3, y - 2, y - 1]
        window_data = []

        for py in prior_years:
            query = f"""
            SELECT TRADE_CODE, DT, NP_BELONGTO_PARCOMSH, TOT_ASSETS, TOT_LIAB
            FROM financialinfo
            WHERE DAY(DT) = 31 AND MONTH(DT) = 3 AND YEAR(DT) = {py}
            """
            df = pd.read_sql(query, connection, parse_dates=['DT'])
            df = df.sort_values(['TRADE_CODE', 'DT'], ascending=[True, False])
            df = df.drop_duplicates(subset=['TRADE_CODE'])
            df = df.dropna(subset=['TOT_ASSETS', 'TOT_LIAB', 'NP_BELONGTO_PARCOMSH'])
            df = df[(df['TOT_ASSETS'] - df['TOT_LIAB']) != 0]
            df['ROE'] = df['NP_BELONGTO_PARCOMSH'] / (df['TOT_ASSETS'] - df['TOT_LIAB']) * 100
            df = df[(df['ROE'] >= -100) & (df['ROE'] <= 100)]
            df['year'] = py
            window_data.append(df[['TRADE_CODE', 'ROE', 'year']])

        merged = pd.concat(window_data)
        agg = merged.groupby('TRADE_CODE')['ROE'].agg(
            ROE_3Y_AVG='mean',
            ROE_3Y_STD='std'
        ).reset_index()
        agg['year'] = y
        roe_by_year.append(agg)

    return pd.concat(roe_by_year).sort_values(['TRADE_CODE', 'year']).reset_index(drop=True)


In [ ]:
years = [2025]
all_samples = build_financial_samples_q(years, connection)

# 已经有 all_samples（从前面的 build_financial_samples 得到）
all_samples_selected = filter_by_market_cap(all_samples)

# 计算 3 年滚动 ROE
final_roe = calculate_3y_rolling_roe_q(years, connection)

# 清洗 ROE 平均值
cleaned_roe, lower, upper = clean_roe_by_iqr(final_roe)

# 获取稳定性阈值
stability_threshold = compute_roe_stability_threshold(cleaned_roe)
merged_data = merge_roe_to_samples(all_samples_selected, cleaned_roe)
print(merged_data.head())
merged_class = classify_and_label_stocks(merged_data, stability_threshold)
merged_0 = merged_class[merged_class['label'] == 0]
merged_1 = merged_class[merged_class['label'] == 1]

In [ ]:
# def build_quarterly_financial_yoy(trade_code: str, analysis_date: str, connection):
#     """
#     获取指定股票在某季度的财务分析结果（仅含同比分析）
    
#     参数：
#         trade_code: 股票代码，如 '600276.SH'
#         analysis_date: 季度日期字符串，如 '2025-03-31'
#         connection: SQLAlchemy 数据库连接对象
        
#     返回：
#         DataFrame：一行，包含财务比率、原始值、同比增长率
#     """
#     import pandas as pd
#     import numpy as np

#     # 解析分析日期 & 同期日期
#     analysis_date = pd.to_datetime(analysis_date)
#     last_year_same_day = analysis_date - pd.DateOffset(years=1)

#     # 格式化为字符串以匹配数据库
#     curr_dt = analysis_date.strftime('%Y-%m-%d')
#     last_dt = last_year_same_day.strftime('%Y-%m-%d')

#     # 拉取两个季度的财报数据
#     cols_str = ', '.join(SELECTED_COLS)
#     query = f"""
#     SELECT {cols_str}
#     FROM financialinfo
#     WHERE TRADE_CODE = '{trade_code}'
#       AND DT IN ('{curr_dt}', '{last_dt}')
#     """
#     df = pd.read_sql(query, connection, parse_dates=['DT'])

#     if df.empty or len(df) < 1:
#         raise ValueError(f"未找到 {trade_code} 在 {curr_dt} 或 {last_dt} 的财务数据")

#     # 计算财务比率
#     df = compute_financial_features(df)

#     # 当前期
#     df_curr = df[df['DT'] == pd.Timestamp(curr_dt)].copy()
#     if df_curr.empty:
#         raise ValueError(f"未找到当前季度数据：{curr_dt}")
#     row = df_curr.iloc[0].copy()

#     # 同期
#     df_last = df[df['DT'] == pd.Timestamp(last_dt)].copy()
#     if not df_last.empty:
#         last_row = df_last.iloc[0]
#         # 比率差值
#         for col in FEATURE_COLS:
#             curr_val = row.get(col)
#             prev_val = last_row.get(col)
#             if pd.notna(curr_val) and pd.notna(prev_val):
#                 row[f"{col}_DIFF"] = curr_val - prev_val
#             else:
#                 row[f"{col}_DIFF"] = np.nan

#         # 原值同比
#         for col in GROWTH_COLS:
#             curr_val = row.get(col)
#             prev_val = last_row.get(col)
#             if pd.notna(curr_val) and pd.notna(prev_val) and prev_val != 0:
#                 row[f"{col}_YOY"] = (curr_val - prev_val) / abs(prev_val)
#             else:
#                 row[f"{col}_YOY"] = np.nan
#     else:
#         for col in FEATURE_COLS:
#             row[f"{col}_DIFF"] = np.nan
#         for col in GROWTH_COLS:
#             row[f"{col}_YOY"] = np.nan


#     # 可选字段重命名
#     rename_cols = {
#         'CASH_PAY_ACQ_CONST_FIOLTA': '资本开支',
#         'RESEARCHANDDEVELOPMENTEXPENSES': '研发费用',
#         'CASH_PAY_ACQ_CONST_FIOLTA_YOY': '资本开支_GROWTH',
#         'RESEARCHANDDEVELOPMENTEXPENSES_YOY': '研发费用_GROWTH'
#     }
#     row = row.rename(rename_cols)

#     return pd.DataFrame([row])


In [ ]:
# result['PE_TTM_PCT_RANK'] = 0.71
# result['is_high_valued'] = True
# result['is_high_valued_growth_stock'] = True
# result['label'] = 1


In [ ]:
def merge_meeting_content_batch_fast(filtered_df: pd.DataFrame, connection) -> pd.DataFrame:
    """
    批量合并 report_extract_text 与  的内容，提高效率。
    与原始 merge_meeting_content_exaccninfotly_final 功能等价但速度更快。

    参数：
        filtered_df: 包含 TRADE_CODE 和 sample_year 的 DataFrame
        connection: SQLAlchemy 连接对象

    返回：
        包含 combined_content 的 DataFrame
    """
    import pandas as pd
    import re

    df = filtered_df.copy()
    df['stock_code'] = df['TRADE_CODE'].str.replace(r'\.(SH|SZ|BJ|HK)$', '', regex=True).str.strip()
    df['start_year'] = df['sample_year'].str.extract(r'(\d{4})').astype(int)

    stock_codes = df['stock_code'].unique().tolist()

    # ========== 批量获取报告摘要 ==========
    report_query = f"""
        SELECT stock_code, extract_text, report_title, publish_time
        FROM report_extract_text
        WHERE stock_code IN ({','.join(f"'{code}'" for code in stock_codes)})
    """
    report_data = pd.read_sql(report_query, connection)

    # ========== 批量获取会议纪要 ==========
    meeting_query = f"""
        SELECT stock_code, meeting_content, article_title
        FROM cninfo
        WHERE stock_code IN ({','.join(f"'{code}'" for code in stock_codes)})
    """
    meeting_data = pd.read_sql(meeting_query, connection)

    # 结果存储
    combined_contents = []

    for idx, row in df.iterrows():
        code = row['stock_code']
        year = row['start_year']

        # 提取对应报告
        rdata = report_data[report_data['stock_code'] == code]
        r_filtered = rdata[rdata['report_title'].str.contains(str(year), case=False, na=False)]
        if not r_filtered.empty:
            latest_report = r_filtered.loc[r_filtered['publish_time'].idxmax()]
            extract_text = latest_report['extract_text']
        else:
            extract_text = ' '

        # 提取对应会议纪要
        mdata = meeting_data[meeting_data['stock_code'] == code]
        m_filtered = mdata[mdata['article_title'].str.contains(str(year), case=False, na=False)]
        if not m_filtered.empty:
            all_meeting_content = ' | '.join(m_filtered['meeting_content'].fillna(''))
        else:
            all_meeting_content = ' '

        combined = (extract_text or '') + ' ' + (all_meeting_content or '')
        combined_contents.append(combined)

    df['combined_content'] = combined_contents

    # 统计输出
    print(f"\n🎯 处理完成 {len(df)} 条样本")
    print(f"   - 非空内容数量: {(df['combined_content'].str.len() > 2).sum()}")
    print(f"   - 覆盖率: {(df['combined_content'].str.len() > 2).mean() * 100:.1f}%")

    return df


In [ ]:
merged_final_1 = merge_meeting_content_batch_fast(merged_1, connection)
merged_final_0 = merge_meeting_content_batch_fast(merged_0, connection)
# 第二步：拼接合并结果
merged_final = pd.concat([merged_final_1, merged_final_0], ignore_index=True)

# 第三步：筛选 combined_content 非空的行
# 条件可以更严格，比如内容长度 > 5
non_empty_rows = merged_final[merged_final['combined_content'].str.strip().str.len() > 0]

# 提取有效的 TRADE_CODE 列表（去重）
valid_trade_codes = non_empty_rows['TRADE_CODE'].dropna().unique().tolist()

# 输出结果
print(f"✅ 共找到 {len(valid_trade_codes)} 个 combined_content 非空的股票代码：")
print(valid_trade_codes)

In [ ]:
from typing import List
import pandas as pd
import numpy as np

def build_quarterly_financial_yoy_batch(trade_codes: List[str], analysis_date: str, connection):
    """
    批量获取多个股票在某季度的财务分析结果（同比分析）

    参数：
        trade_codes: 股票代码列表，如 ['600276.SH', '000858.SZ']
        analysis_date: 季度日期字符串，如 '2025-03-31'
        connection: SQLAlchemy 数据库连接对象

    返回：
        DataFrame：每个股票一行，包含财务比率、原始值、同比增长率
    """

    # 解析分析日期 & 同期日期
    analysis_date = pd.to_datetime(analysis_date)
    last_year_same_day = analysis_date - pd.DateOffset(years=1)

    # 格式化为字符串
    curr_dt = analysis_date.strftime('%Y-%m-%d')
    last_dt = last_year_same_day.strftime('%Y-%m-%d')

    # SQL 查询
    code_list_str = ', '.join(f"'{code}'" for code in trade_codes)
    cols_str = ', '.join(SELECTED_COLS)

    query = f"""
    SELECT {cols_str}
    FROM financialinfo
    WHERE TRADE_CODE IN ({code_list_str})
      AND DT IN ('{curr_dt}', '{last_dt}')
    """
    df = pd.read_sql(query, connection, parse_dates=['DT'])

    if df.empty:
        raise ValueError(f"未找到任何股票在 {curr_dt} 或 {last_dt} 的财务数据")

    # 计算财务比率（假设 compute_financial_features 已定义）
    df = compute_financial_features(df)

    results = []

    for code in trade_codes:
        df_sub = df[df['TRADE_CODE'] == code]
        df_curr = df_sub[df_sub['DT'] == pd.Timestamp(curr_dt)].copy()
        df_last = df_sub[df_sub['DT'] == pd.Timestamp(last_dt)].copy()

        if df_curr.empty:
            continue  # 当前数据不存在，跳过
        row = df_curr.iloc[0].copy()

        # 同比计算
        if not df_last.empty:
            last_row = df_last.iloc[0]
            for col in FEATURE_COLS:
                row[f"{col}_DIFF"] = row.get(col) - last_row.get(col) if pd.notna(row.get(col)) and pd.notna(last_row.get(col)) else np.nan
            for col in GROWTH_COLS:
                curr_val = row.get(col)
                prev_val = last_row.get(col)
                if pd.notna(curr_val) and pd.notna(prev_val) and prev_val != 0:
                    row[f"{col}_YOY"] = (curr_val - prev_val) / abs(prev_val)
                else:
                    row[f"{col}_YOY"] = np.nan
        else:
            for col in FEATURE_COLS:
                row[f"{col}_DIFF"] = np.nan
            for col in GROWTH_COLS:
                row[f"{col}_YOY"] = np.nan

        # 字段重命名
        rename_cols = {
            'CASH_PAY_ACQ_CONST_FIOLTA': '资本开支',
            'RESEARCHANDDEVELOPMENTEXPENSES': '研发费用',
            'CASH_PAY_ACQ_CONST_FIOLTA_YOY': '资本开支_GROWTH',
            'RESEARCHANDDEVELOPMENTEXPENSES_YOY': '研发费用_GROWTH'
        }
        row = row.rename(rename_cols)

        results.append(row)

    return pd.DataFrame(results)


In [ ]:
trade_codes = valid_trade_codes
#trade_codes: 股票代码列表，如 ['600276.SH', '000858.SZ']
# trade_codes = ['600276.SH']
analysis_date = '2025-03-31'

result = build_quarterly_financial_yoy_batch(trade_codes, analysis_date, connection)


In [ ]:
final_result = result.merge(non_empty_rows, on='TRADE_CODE', how='inner')

In [ ]:
# 导入 Excel 文件
stock_class = pd.read_excel('股票分类归因.xlsx')

# 查看导入的数据
print(stock_class.head())  # 输出前几行数据


In [ ]:
final_result

In [ ]:
import json
from openai import OpenAI

MODEL = "anthropic/claude-sonnet-4"

# 创建 OpenAI 客户端
client = OpenAI(
    api_key='',
    base_url=''
)

# 构建系统提示
system_prompt = """你是一个金融分析师。你可以分析股票的财务数据，并判断股票的表现是否符合上涨或下跌的共性特征。"""

# AI 请求函数
def ask_ai(question):
    # 发送请求
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]
    )
    # 获取返回结果
    res = response.choices[0].message.content
    return res

# 创建字典存储每个股票类型的上涨和下跌共性
common_up = {
    '高估值成长股': [],
    '低估值价值股': []
}

common_down = {
    '高估值成长股': [],
    '低估值价值股': []
}

# 根据股票分类和上涨/下跌，分类共性
for index, row in stock_class.iterrows():
    stock_type = row['股票分类']
    trend = row['上涨or下跌']
    feature = row['共性']
    
    if trend == '上涨':
        common_up[stock_type].append(feature)
    elif trend == '下跌':
        common_down[stock_type].append(feature)



In [ ]:
up_features = common_up.get(stock_type, [])
down_features = common_down.get(stock_type, [])

In [ ]:
up_features

In [ ]:
# 创建一个列表来保存所有股票的分析结果
ai_responses = []

# 获取 final_results_df 中的每一行，并根据分类进行分析
for index, row in final_result.iterrows():
    # 提取所有相关列的信息
    stock_code = row['TRADE_CODE']
    label = row['label']
    if label == 0:
        stock_type = '低估值价值股'
    elif label == 1:
        stock_type = '高估值成长股'
    else:
        stock_type = '未分类'
    
    # 分类标志
    label = row['label']

    
    # 内容信息
    combined_content = row['combined_content']
    
    # 获取对应股票分类的上涨和下跌共性
    up_features = common_up.get(stock_type, [])
    down_features = common_down.get(stock_type, [])
    
    # 构建问题，询问 AI 是否符合上涨或下跌的共性
    question_1 = f"""
    请你按照以下格式严格输出分析结果：

    1. 股票代码：{stock_code}
    2. 股票名称
    3. 股票类型（高估值成长股 / 低估值价值股）：{stock_type}

    **详细财务数据表格分析：**
    {final_result}

    请根据该股票的财务数据，判断它属于以下哪个行业：
    \n\n{output}\n\n

    **财务特征分析：**
    基于上述信息以及详细财务数据表格，该股票的关键财务数据包括哪些？请精确给出数据：

    **综合内容分析：**
    {combined_content}

    **特征匹配分析：**
    对照下面两类共性特征，根据综合内容、财务特征、财务数据表等上述内容，**逐条判断每条特征是否符合**，并给出结论：
    
    符合上涨趋势的共性特征（{up_features}）：
    请逐条分析每一条特征是否符合，结合上述财务数据和内容信息进行判断。
    
    符合下跌趋势的共性特征（{down_features}）：
    请逐条分析每一条特征是否符合，结合上述财务数据和内容信息进行判断。

    **统计总结：**
    - 符合上涨特征数量：
    - 符合下跌特征数量：

    **投资建议：**
    最后，请基于这些信息，单纯从指定日期 {analysis_date} 开始，判断该股票在接下来的多久时间内，更可能是上涨还是下跌？请说明判断依据。
    """

    # 发送问题并获取 AI 的回应
    ai_response_1 = ask_ai(question_1)
    
    # 将结果存储在 ai_responses 列表中，每个元素是一个字典
    ai_responses.append({
        'stock_code': stock_code,
        'stock_type': stock_type,
        'label': label,
        'analysis_result': ai_response_1
    })

# 输出所有股票的分析结果
for response in ai_responses:
    print(f"分析结果 for stock {response['stock_code']} ({response['stock_type']}, 标签: {response['label']}):")
    print(response['analysis_result'])
    print("\n" + "="*50 + "\n")

In [ ]:
#简易版 但是需要小于200000个tokens
question_2 = f"对该答案进行各股票涨跌部分的简要概括：\n{ai_responses}\n，并且根据实际交易情况验证一下你判断的准确率"
ai_response_2 = ask_ai(question_2)
print(ai_response_2)

In [ ]:
import tiktoken

# 高效的切片函数（仅在需要时切）
def split_text_by_tokens(text, max_tokens=120000):
    enc = tiktoken.get_encoding("cl100k_base")
    tokens = enc.encode(text)
    if len(tokens) <= max_tokens:
        return [text]  # 不切
    return [enc.decode(tokens[i:i + max_tokens]) for i in range(0, len(tokens), max_tokens)]

# 主处理函数：切片+每段调用ask_ai
def summarize_long_text(ai_responses, ask_fn):
    # 🚦 自动处理非字符串输入
    if isinstance(ai_responses, list):
        ai_responses = "\n".join(str(x) for x in ai_responses)
    elif not isinstance(ai_responses, str):
        ai_responses = str(ai_responses)

    # ✂️ 切片（仅当必要时切）
    chunks = split_text_by_tokens(ai_responses)

    summaries = []
    for i, chunk in enumerate(chunks):
        if not chunk.strip():  # 🚫 跳过空白内容
            continue
        prompt = f"请从下面的内容中提取每只股票的名称、预测涨跌方向，并列出：\n{chunk}"
        try:
            response = ask_fn(prompt)
            summaries.append(response)
        except Exception as e:
            print(f"第{i+1}段处理失败: {e}")
    
    return summaries

# 汇总总结
def summarize_all(ai_responses, ask_fn):
    summaries = summarize_long_text(ai_responses, ask_fn)
    if not summaries:
        return "没有有效内容可总结。"
    final_prompt = (
    "以下是多个段落中提取出来的股票预测情况：\n\n"
    + "\n\n".join(summaries)
    + "\n\n请你将这些信息进行整合，不分批，所有提到的股票的数据都要列出，格式如下：\n"
    + "证券代码 + 股票名 + 预测涨跌\n"
)
    return ask_fn(final_prompt)


result = summarize_all(ai_responses, ask_ai)
print(result)

